In [2]:
from arraytainers import Numpytainer, Jaxtainer
import numpy as np
from numpy import array
import jax.numpy as jnp
import functools
from jax.numpy import DeviceArray

In [16]:
# a = [{'a':(1,2), 'b':[(2,3)]}]
a = {'a':{'c':np.ones((3,2)), 'd':np.ones((2,3))}, 'b':[{'b':np.ones((3,3))}, np.ones((1,9,1))]}
b = {'a':(6,1), 'b':(1,1,9)} 
A = Numpytainer(a)
B = Numpytainer(b)
A.reshape(B).shape

Numpytainer({'a': {'c': array([6, 1]), 'd': array([6, 1])}, 'b': [{'b': array([1, 1, 9])}, array([1, 1, 9])]})

In [20]:
a = {'a':{'c':np.ones((3,2)), 'd':np.ones((3,2))}, 'b':[{'b':np.ones((4,4))}, np.ones((4,4))]}
b = {'a':np.ones((3,1)), 'b':np.ones((1,4))}
A = Numpytainer(a)
B = Numpytainer(b)
(A + B).shape

Numpytainer({'a': {'c': array([3, 2]), 'd': array([3, 2])}, 'b': [{'b': array([4, 4])}, array([4, 4])]})

In [22]:
a = {'a':{'c':np.ones((3,2)), 'd':np.ones((3,2))}, 'b':[{'b':np.ones((4,4))}, np.ones((4,4))]}
b = {'a':np.ones((2,3)), 'b':np.ones((4,6))}
A = Numpytainer(a)
B = Numpytainer(b)
(A @ B).shape

Numpytainer({'a': {'c': array([3, 3]), 'd': array([3, 3])}, 'b': [{'b': array([4, 6])}, array([4, 6])]})

In [11]:
# b = [{'a':np.ones((1,2)), 'b':[np.ones((2,3))]}]
b = [{'a':np.ones((1,2,3,4)), 'b':[np.ones((2,3,4,5))]}]
np.append(b, np.product(Numpytainer(b).shape[1:]))

Numpytainer([{'a': array([{'a': array([[[[1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.]],

               [[1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.]]]]), 'b': [array([[[[1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.]],

               [[1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.]],

               [[1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.]]],


              [[[1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.]],

               [[1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1.],
                [1., 1., 1., 1., 1

In [5]:
from copy import deepcopy

In [7]:
deepcopy(jnp.ones((1,2)))

array([[1., 1.]], dtype=float32)

In [5]:
A.contents

[Jaxtainer({'a': DeviceArray([[1., 1.]], dtype=float32), 'b': DeviceArray([[1., 1., 1.],
              [1., 1., 1.]], dtype=float32)})]

In [9]:
A = Jaxtainer([{'a':np.ones((1,2)), 'b':[np.ones((2,3))]}])
B = Jaxtainer([{'a':(1,2), 'b':[(2,3)]}])
assert_equal_values(A.shape.unpacked, B.unpacked, approx_equal=True)

[1 2] [1 2]
True
[2 3] [2 3]
True


In [4]:
def assert_equal_values(contents_1, contents_2, approx_equal=True):
  
    def assert_func(contents_1, contents_2):
        print(contents_1, contents_2)
        if approx_equal:
          print(np.allclose(contents_1, contents_2, atol=1e-5, rtol=1e-5))
            # Need to adjust atol and rtol because, by default, Jax uses 32 bit numbers which means,
            # when compared to 64 bit Numpy computations, the default tolerances throw spurious errors:
          assert np.allclose(contents_1, contents_2, atol=1e-5, rtol=1e-5)
        else:
          assert contents_1 == contents_2
    apply_func_to_contents(contents_1, contents_2, func=assert_func, throw_exception=True)


def apply_func_to_contents(*contents, func=None, args=(), kwargs=None, index_args=None, 
                            return_list=False, throw_exception=False):

    try:
        expected = call_func_recursive(contents, func, args, kwargs, index_args, return_list)
        exception = None
    except Exception as error:
        if throw_exception:
            raise error
        else:
            expected = None
            exception = error

    return (expected, exception)

def call_func_recursive(contents, func, args, kwargs, index_args, return_list):
    
    first_item = contents[0]
    if kwargs is None:
      kwargs = {}

    if index_args is None:
        index_args = lambda args, idx : args

    if isinstance(first_item, dict):
        contents_i = {key: [item_i[key] for item_i in contents] for key in first_item.keys()}
        result = {key: call_func_recursive(contents_i[key], func, index_args(args,key), kwargs, index_args, return_list) 
                  for key in first_item.keys()}
        return list(result.values()) if return_list else result

    elif isinstance(first_item, (tuple,list)):
        contents_i = [[item_i[idx] for item_i in contents] for idx in range(len(first_item))]
        return [call_func_recursive(contents_i[idx], func, index_args(args,idx), kwargs, index_args, return_list)  
                for idx in range(len(first_item))]

    else:
        return func(*contents, *args, **kwargs)

In [7]:
Jaxtainer({}).contents == Jaxtainer({}).shape.contents

True

In [2]:
# a = [array([[3.74540119, 9.50714306, 7.31993942],
#        [5.98658484, 1.5601864 , 1.5599452 ],
#        [0.58083612, 8.66176146, 6.01115012]]), array([[3.74540119, 9.50714306],
#        [7.31993942, 5.98658484]])]
# key = array([[ 0, -1],[ 0, -1]])
a = [jnp.array([[3.74540119, 9.50714306, 7.31993942],
       [5.98658484, 1.5601864 , 1.5599452 ],
       [0.58083612, 8.66176146, 6.01115012]]), jnp.array([[3.74540119, 9.50714306],
       [7.31993942, 5.98658484]])]
key = jnp.array([10])

In [3]:
a = {'a':[[1, 2], [1,2]], 'b': np.ones((3,3,3))}
B = Numpytainer(a)

In [6]:
B = Numpytainer([{'a':np.ones((1,2)),'b':np.ones((2,3))}, {'b':np.ones((2,3)),'c':np.ones((4,)),'d':np.ones((5,6))}])

In [16]:
np.ones((4,)).shape[0:1]

(4,)

In [26]:
B.shape

Numpytainer([{'a': [1, 2], 'b': [2, 3]}, {'b': [2, 3], 'c': [4], 'd': [5, 6]}])

In [8]:
A

Numpytainer([{'a': array([1, 2]), 'b': array([2, 3])}, {'b': array([2, 3]), 'c': array([4]), 'd': array([5, 6])}])

In [2]:
# A = Numpytainer([{'a':(1,2),'b':(2,3)}, {'b':(2,3),'c':(4,),'d':(5,6)}])
A = Numpytainer([{'a':np.ones((1,2)),'b':np.ones((2,3))}, {'b':np.ones((2,3)),'c':np.ones((4,)),'d':np.ones((5,6))}])
B = Numpytainer([{'a':(2,),'b':(6,)}, {'b':(6,),'c':(4,),'d':(30,)}])
A.reshape(B).shape

Numpytainer([{'a': array([2]), 'b': array([6])}, {'b': array([6]), 'c': array([4]), 'd': array([30])}])

In [5]:
print(A.shape)
A.flatten().shape

Numpytainer([{'a': array([1, 2]), 'b': array([2, 3])}, {'b': array([2, 3]), 'c': array([4]), 'd': array([5, 6])}])


Numpytainer([{'a': array([2]), 'b': array([6])}, {'b': array([6]), 'c': array([4]), 'd': array([30])}])

In [5]:
np.reshape(array([2, 3]), (6,))

ValueError: cannot reshape array of size 2 into shape (6,)

In [4]:
A = Jaxtainer(a)
B = Numpytainer([(2,2)])

In [5]:
np.array([[1, 2], [1,2]]).shape[1:]

(2,)

In [15]:
B.shape[1:]

Numpytainer({'a': (2,), 'b': (3, 3)})

In [7]:
C = B[key]

IndexError: index 10 is out of bounds for axis 0 with size 2

In [ ]:
C = A[key]

In [ ]:
jnp.array([3.74540119, 9.50714306, 7.31993942])[jnp.array([ 0, -1])]

In [ ]:
index_func = lambda contents : contents[key]
apply_func_to_contents(a, func=index_func, args=(key,))

In [ ]:
def apply_func_to_contents(*contents, func=None, args=(), kwargs=None, index_args=None, 
                            return_list=False, throw_exception=False):

    try:
        expected = call_func_recursive(contents, func, args, kwargs, index_args, return_list)
        exception = None
    except Exception as error:
        if throw_exception:
            raise error
        else:
            expected = None
            exception = error

    return (expected, exception)

def call_func_recursive(contents, func, args, kwargs, index_args, return_list):
    
    first_item = contents[0]
    
    if kwargs is None:
      kwargs = {}

    if index_args is None:
        index_args = lambda args, idx : args

    if isinstance(first_item, dict):
        contents_i = {key: [item_i[key] for item_i in contents] for key in first_item.keys()}
        result = {key: call_func_recursive(contents_i[key], func, index_args(args,key), kwargs, index_args, return_list) 
                  for key in first_item.keys()}
        return list(result.values()) if return_list else result

    elif isinstance(first_item, (tuple,list)):
        contents_i = [[item_i[idx] for item_i in contents] for idx in range(len(first_item))]
        return [call_func_recursive(contents_i[idx], func, index_args(args,idx), kwargs, index_args, return_list)  
                for idx in range(len(first_item))]

    else:
        print(contents, args)
        return func(*contents, *args, **kwargs)